In [1]:
%load_ext autoreload
%autoreload 2

import torch
import pickle
from transformer_lens import HookedTransformer, HookedTransformerConfig
from huggingface_hub import hf_hub_download

from eap.graph import Graph
from eap.attribute import attribute

from MIB_circuit_track.circuit_loading import load_graph_from_json
from MIB_circuit_track.dataset import HFEAPDataset
from MIB_circuit_track.metrics import get_metric

In [2]:
task_name = "ioi"

hf_cfg = hf_hub_download("cybershiptrooper/InterpBench", subfolder=task_name, filename="ll_model_cfg.pkl")
hf_model = hf_hub_download("cybershiptrooper/InterpBench", subfolder=task_name, filename="ll_model.pth")

cfg_dict = pickle.load(open(hf_cfg, "rb"))
if isinstance(cfg_dict, dict):
    cfg = HookedTransformerConfig.from_dict(cfg_dict)
else:
    # Some cases in InterpBench have the config as a HookedTransformerConfig object instead of a dict
    assert isinstance(cfg_dict, HookedTransformerConfig)
    cfg = cfg_dict
cfg.device = "cuda"

if "ioi" in task_name:
    # Small hack to enable evaluation mode in the IOI model, that has a different config during training
    cfg.use_hook_mlp_in = True
    cfg.use_attn_result = True
    cfg.use_split_qkv_input = True

model = HookedTransformer(cfg)
model.load_state_dict(torch.load(hf_model, map_location="cuda"))

ll_model.pth:   0%|          | 0.00/42.4M [00:00<?, ?B/s]

/tmp/ipykernel_963074/162056255.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(hf_model, map_location="cuda"))


<All keys matched successfully>

In [3]:
model = HookedTransformer.from_pretrained("gpt2-small")

Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
model.tokenizer.encode(" Mary")

[5335]

In [5]:
import csv
with open("ioi-gpt2.csv", 'r') as ioi_data, open("ioi-interpbench.csv", 'w') as ioi_out:
    reader = csv.reader(ioi_data)
    writer = csv.writer(ioi_out)
    writer.writerow(["", "clean", "corrupted", "corrupted_hard", "correct_idx", "incorrect_idx"])
    next(reader)
    for row in reader:
        _idx, clean, corrupted, corrupted_hard, _, _ = row
        name1 = clean.split()[1]
        name2 = clean.split()[3]
        name1_count = clean.count(name1)
        name2_count = clean.count(name2)
        if name1_count > 1:
            correct_name = name2
            incorrect_name = name1
        elif name2_count > 1:
            correct_name = name1
            incorrect_name = name2
        else:
            raise Exception(f"Can't find correct name among {name1} and {name2}")
        correct_idx = model.tokenizer.encode(f" {correct_name}")[0]
        incorrect_idx = model.tokenizer.encode(f" {incorrect_name}")[0]
        writer.writerow([_idx, clean, corrupted, corrupted_hard, correct_idx, incorrect_idx])

FileNotFoundError: [Errno 2] No such file or directory: 'ioi-gpt2.csv'

In [ ]:
reference_graph = load_graph_from_json("interpbench/ioi_groundtruth.json")
hypothesis_graph = Graph.from_model(model)

In [ ]:
dataset = HFEAPDataset("danaarad/ioi_dataset", model.tokenizer, task='ioi')
dataloader = dataset.to_dataloader(8)
metric_fn = get_metric("prob_diff", "ioi", model.tokenizer, model)

Filter: 100%|██████████| 100/100 [00:00<00:00, 12527.42 examples/s]


In [ ]:
dataset[5]

('Then, Amber and Bradley were thinking about going to the house. Amber wanted to give a drink to',
 'Then, Amber and Bradley were thinking about going to the house. Bradley wanted to give a drink to',
 [[16182], [21896]])

In [ ]:
hypothesis_graph = Graph.from_model(model)
attribute(model, hypothesis_graph, dataloader, metric_fn, 'EAP')

100%|██████████| 32491/32491 [00:00<00:00, 379228.09it/s]


In [ ]:
hypothesis_graph_ig = Graph.from_model(model)
attribute(model, hypothesis_graph_ig, dataloader, metric_fn, 'EAP-IG-inputs', ig_steps=5)

100%|██████████| 32491/32491 [00:00<00:00, 364502.11it/s]


In [ ]:
sorted(hypothesis_graph_ig.edges.values(), key=lambda x:x.score, reverse=True)

[Edge(m0->logits, score: 0.005371724721044302, in_graph: True),
 Edge(a1.h1->m4, score: 0.0015480746515095234, in_graph: True),
 Edge(m3->m4, score: 0.001099234912544489, in_graph: True),
 Edge(m0->m5, score: 0.000999135198071599, in_graph: True),
 Edge(m0->a4.h3<v>, score: 0.0008317457395605743, in_graph: True),
 Edge(m2->m4, score: 0.0006441010627895594, in_graph: True),
 Edge(a1.h0->m4, score: 0.0004793550760950893, in_graph: True),
 Edge(m0->a1.h2<v>, score: 0.00045301244244910777, in_graph: True),
 Edge(m2->m5, score: 0.00044858353794552386, in_graph: True),
 Edge(a2.h0->a4.h0<v>, score: 0.00044200217234902084, in_graph: True),
 Edge(a0.h3->logits, score: 0.00043037495925091207, in_graph: True),
 Edge(a2.h2->a4.h0<v>, score: 0.0004138057411182672, in_graph: True),
 Edge(a2.h3->a5.h0<v>, score: 0.0004055552708450705, in_graph: True),
 Edge(m3->logits, score: 0.00040196688496507704, in_graph: True),
 Edge(a2.h1->a4.h0<v>, score: 0.00039761303924024105, in_graph: True),
 Edge(a2.h3->

In [ ]:
hypothesis_graph.to_json("circuits/ioi_prob_diff_vanilla_gpt2.json")
hypothesis_graph_ig.to_json("circuits/ioi_prob_diff_ig_gpt2.json")

In [ ]:
import random

random_graph = Graph.from_model(model)
for edge in random_graph.edges.values():
    edge.score = random.uniform(-1.0, 1.0)
random_graph.edges.values()
random_graph.to_json("circuits/ioi_random_interpbench.json")